This scripts expects the following folder organization:
- home_folder
    - input1_folder: contains: image files; PVN rois files. Image files are tiff files with dimensions ch, x, y. Ch ('channel') has Nuclei (DAPI) at 4th position. PVN rois files are .zip or .roi Java-based roi files (drawn in ImageJ); their names match the corresponding Image file (but for the extension).
    - input2_folder: contains Cells roi files. Cells rois are .zip Java-based roi files (drawn in ImageJ). Their names match the corresponding Image files, but for the extension. Per each cell there is a 'Nucleus' and a 'Cell' roi, which is the expanded nucleus from pipeline step-3.

Outputs are saved in an output_folder inside home_folder

NOTES: 1) by default the scripts expects to find the following information in the file directory: experiment name; mouse name; sex; slice position. 2) This scripts works on the pipeline step-3 results.

In [1]:

import numpy as np
import tifffile
import sklearn
import matplotlib.pyplot as plt
import os
import pandas as pd
import skimage
from skimage import exposure
from skimage.io import imread, imshow
from skimage.color import rgb2gray, rgb2hsv
from skimage.measure import label, regionprops, regionprops_table
from skimage.util import img_as_int, img_as_uint, img_as_float, img_as_float32, img_as_ubyte
from skimage.feature import blob_dog, blob_log, blob_doh

from findmaxima2d import find_maxima, find_local_maxima #Version that has been installed into python site-packages.
import czifile
from read_roi import read_roi_zip
import roifile
from roifile import ImagejRoi
from skimage.draw import polygon

import random




In [2]:
#Indicate the directory of the home_folder
home_directory = r"/Users/alessandro_ulivi/ownCloud - alessandro.ulivi@lin-magdeburg.de@owncloud.gwdg.de/RNAscope/Analysis/Analysis_directory/Experiment2"

#Join home_directory with input1_folder (Step_1_output), input2_folder (Step_3_output) and output_folder (Step_4_output)
step1_path_output = os.path.join(home_directory, "Step_1_output")
#step2_path_output = os.path.join(home_directory, "Step_2_output")
step3_path_output = os.path.join(home_directory, "Step_3_output")
step4_path_output = os.path.join(home_directory, "Step_4_output")

#Create output_folder, if it doesn't exist
if not os.path.exists(step4_path_output):
    os.makedirs(step4_path_output)


#Generates a list of the files in a given directory avoiding hidden files
def listdirNHF(path100):
    """
    forms a list of files in an directory, avoiding hidden files. Hidden files are identified by their names starting with a '.'
    inputs: folder directory
    outputs: list of all non-hidden files in the input directory
    """
    #Initialize output list
    the_F_list = []
    
    #Collect input folder elements in a list
    start_list = os.listdir(path100)
    
    #Iterate through the elements of the list
    for item in start_list:
        
        #If the name of the element doesn't start with a '.', append it to the output list
        if not item.startswith('.'):
            the_F_list.append(item)
    
    #Return the output list
    return the_F_list



In [3]:
#define hyperparameters for the analysis

#histogram rescaling - set bottom and top percentils of signal intensity, to use when defining the rescaling range
min_percentil = 0.1 #NOTE: this is irrelevant for the processing
max_percentil = 99.995 #Parameter used in the analysis is 99.995


#blob detection
min_sigma_Ch1=1 #lower sigma min means detecting smaller blobs; ref to https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.blob_log
max_sigma_Ch1=8 #higher sigma max means detecting bigger blobs; ref to https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.blob_log
num_sigma_Ch1=4 #ref to https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.blob_log
threshold_Ch1=0.03 #Intensity threshold to detect blobs; used in the "threshold" parameter of the blob_log function
overlap_Ch1=0.9 #when two blobs' areas overlap more than the specified threshold, the smallest is removed

min_sigma_Ch2=1 #lower sigma min means detecting smaller blobs; ref to https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.blob_log
max_sigma_Ch2=8 #higher sigma max means detecting bigger blobs; ref to https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.blob_log
num_sigma_Ch2=4 #ref to https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.blob_log
threshold_Ch2=0.02 #Intensity threshold to detect blobs; used in the "threshold" parameter of the blob_log function
overlap_Ch2=0.9 #when two blobs' areas overlap more than the specified threshold, the smallest is removed

min_sigma_Ch3=1 #lower sigma min means detecting smaller blobs; ref to https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.blob_log
max_sigma_Ch3=8 #higher sigma max means detecting bigger blobs; ref to https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.blob_log
num_sigma_Ch3=4 #ref to https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.blob_log
threshold_Ch3=0.03 #Intensity threshold to detect blobs; used in the "threshold" parameter of the blob_log function
overlap_Ch3=0.9 #when two blobs' areas overlap more than the specified threshold, the smallest is removed


#find maxima
maxima_threshold_Ch1 = 15
maxima_threshold_Ch2 = 12
maxima_threshold_Ch3 = 15

#Names
experiment_position = home_directory[-1:] #Specify how to extract the experiment information. If there is no experiment information, indicate any string
sex_position = [None,1] #Specify how to extract the sex information. In this case, the list indicates the start and end of Image_name slicing (None is used to start from the beginning of Image_name). If there is no sex information, indicate any string.
mouse_position = [2,4] #Specify how to extract the mouse information. In this case, the list indicates the start and end of Image_name slicing. If there is no mouse information, indicate any string
slice_position = [7,11] #Specify how to extract the slice_position information. In this case, the list indicates the start and end of Image_name slicing. If there is no sex information, indicate any string



In [4]:
#organize files in a dictionary: each "Image file name" (string without extension) is linked to a dictionary_2.
#Dictionary_2 links: 1) 'original_name' to "Image file name"+.tif extension;  2) 'rois_pvn' to "Image file name"+.zip or .roi extension; 3) 'cell_rois' to "Image file name"+"_expanded"+.zip extension 

organized_dictionary = {}
for f in listdirNHF(os.path.join(home_directory, step1_path_output)):
    filename = str(f)[:-4]
    if f[-4:]==".tif":
        organized_dictionary[filename]={}
        organized_dictionary[filename]['original_image']= str(f)
        pvn_rois_name = filename + ".zip"
        if os.path.exists(os.path.join(os.path.join(home_directory, step1_path_output),pvn_rois_name)):
            organized_dictionary[filename]['rois_pvn'] = pvn_rois_name
        else:
            pvn_rois_name2 = filename + ".roi"
            organized_dictionary[filename]['rois_pvn'] = pvn_rois_name2
        cell_rois_name = filename+"_expanded.zip"
        organized_dictionary[filename]['cell_rois'] = cell_rois_name

# print(organized_dictionary)


In [5]:

#functions for processing

def form_polygon(original_picture, my_roi_coordinates):
    """
    inputs: rois coordinates (list of tuples, each tuple is: col, row) and 2D image
    oputputs: mask (image with same shape as input, but with 1 in the rois pixels, 0 otherwise), list of row coordinates, list of col coordinates
    """
    list_x_col = []
    list_y_row = []
    for c in my_roi_coordinates:
        list_x_col.append(c[0]-1)
        list_y_row.append(c[1]-1)
    
    arr_x_col = np.asarray(list_x_col)
    arr_y_row = np.asarray(list_y_row)
    
    img = np.zeros((original_picture.shape[0], original_picture.shape[1]), dtype=np.uint8)

    rr, cc = polygon(arr_y_row, arr_x_col)
    img[rr, cc] = 1
    
    return img, rr, cc


def get_my_mode_value(arr_2D):
    """
    given a 2D numpy array, returns a single mode value into a list. The value is calculated as following: 1) Calculate a 40000-bins histogram distribution; 2) each bin count is defined as a "spike" if its delta change from previous and following bin counts are higher than, respectively: (mean of previous 3 delta changes + 15*(standard deviation of previous 3 delta changes)) and (mean of following 3 delta changes + 15*(standard deviation of following 3 delta changes))
    3) the mode value is the bin which as the highest counts and is not a spike.
    """
    hist, bin_edges = np.histogram(arr_2D, bins=np.linspace(0,65535,40000))
    list_counts = list(hist.ravel())
    
    step_forw = 3
    step_back = 3
    std_numb = 15
    
    spikes = []
    i=0
    for q in range(len(list_counts)):
        if i<step_forw:
            following_deltas = [abs(j-list_counts[list_counts.index(j)+1]) for j in list_counts[i+1:i+step_forw]]
            previous_deltas = [abs(j-list_counts[list_counts.index(j)-1]) for j in list_counts[1:i]]
            total_deltas = following_deltas + previous_deltas
        
        elif i>(len(list_counts)-4):
            following_deltas = [abs(j-list_counts[list_counts.index(j)+1]) for j in list_counts[i+1:-1]]
            previous_deltas = [abs(j-list_counts[list_counts.index(j)-1]) for j in list_counts[i-step_back:i]]
            total_deltas = following_deltas + previous_deltas
        else:
            following_deltas = [abs(j-list_counts[list_counts.index(j)+1]) for j in list_counts[i+1:i+step_forw]]
            previous_deltas = [abs(j-list_counts[list_counts.index(j)-1]) for j in list_counts[i-step_back:i]]
            total_deltas = following_deltas + previous_deltas
        
        mean_delta, std_delta = np.mean(total_deltas), np.std(total_deltas)
        
        if i==0:
            follow_i = list_counts[i+1]
            delta_follow = abs(list_counts[i]-follow_i)
            if delta_follow>(mean_delta+15*std_delta):
                spikes.append(list_counts[i])
            
        elif i==(len(list_counts)-1):
            previous_i = list_counts[i-1]
            delta_previous = abs(list_counts[i]-previous_i)
            if delta_previous>(mean_delta+15*std_delta):
                spikes.append(list_counts[i])
            
        else:
            follow_i = list_counts[i+1]
            previous_i = list_counts[i-1]
            delta_follow = abs(list_counts[i]-follow_i)
            delta_previous = abs(list_counts[i]-previous_i)
            if ((delta_follow>(mean_delta+std_numb*std_delta)) and (delta_previous>(mean_delta+std_numb*std_delta))):
                spikes.append(list_counts[i])
        
        i = i + 1
    
    sorted_list_counts = sorted(list_counts)
    k = 1
    mode_count = sorted_list_counts[-k]
    while (mode_count in spikes):
        mode_count = sorted_list_counts[-k]
        k=k+1
    
    mode_idx_2Darr = np.argwhere(hist == mode_count)
    mode_val_2Darr = bin_edges[mode_idx_2Darr].flatten().tolist()
    if len(mode_val_2Darr)>1:
        mode_val_2Darr = [np.mean(mode_val_2Darr)]
    return mode_val_2Darr



def my_intensity_rescaling(img2rescale, my_min_perc, my_max_perc):
    """
    given a 2D numpy array, a min and a max percentil (integers or floats)
    calculates the intensity values corresponding to the indicated min and max percentiles in the intensity histogram
    calculates the mode value of the intensity histogram (using get_my_mode_value function)
    rescales the intensity values of the 2D array withing a new range defined by the mode and max (derived from the percentiles) intensity values
    returns the a new 2D array, identical to the input but with rescaled intensity values
    """
    #find arrays modes
    mode_val_I2R = get_my_mode_value(img2rescale)[0]

    #find arrays percentiles
    min_I2R, max_I2R = np.percentile(img2rescale, (my_min_perc, my_max_perc))

    #equalize histograms
    equalized_img_I2R = exposure.rescale_intensity(img2rescale, in_range=(mode_val_I2R, max_I2R))
    
    return equalized_img_I2R


def my_find_maxima(file_to_analyze, my_ntol):
    """
    input: 2D array (dtype int16, value range 0,255); threshold
    output: coordinates of local intensity maxima (based on the python version of ImageJ "Find Maxima" https://github.com/dwaithe/MaximaFinder)
    """
    local_max = find_local_maxima(file_to_analyze)
    y, x, regs = find_maxima(file_to_analyze,local_max,my_ntol)
    return y, x, regs



def GetCiPVNwIntMaxBlobs(rescaled_Ch1_Ch2_Ch3, pvn__rois, rois_file, blobs_results, maxima_results, pvn_zip_files):
    """
    GetCiPVNwIntMaxBlobs stays for "Get Cells in the PVN with their Intensities, Maxima and Blobs"
    inputs: list of three 2D arrays to be analyzed
            roifile of the pvn regions. Java-based roi file manually drawn on ImageJ
            roifile of cells, nuclei and cytoplasms in the 2D arrays to analyze. One file is common to all three arrays. This is a Java-based roi file and is the result of pipeline step-3 (ImageJ expansion script). Cells names format is "Cell #XXX"; nuclei names format is "Nucleus #XXX"; cytoplasms names format is "Cytoplasm #XXX".
            results of blobs detection from skimage.feature blob_log function in a list-like object. One result should be provided per each 2D array to analyze
            results of maxima detection from my_find_maxima function in a list-like object. One result should be provided per each 2D array to analyze
    outputs: Tuple of following measurments (output_position,output_measurement, output_format):
    # 0 blobs_in_PVNcells_yx_rrcc_final_dict_Ch1; {"c1":[(y_rr,x_cc,radius),(y_rr,x_cc,radius)], "c2":[(y_rr,x_cc,radius), (y_rr,x_cc,radius)]}
    # 1 blobs_in_PVNcells_yx_rrcc_final_dict_Ch2; {"c1":[(y_rr,x_cc,radius),(y_rr,x_cc,radius)], "c2":[(y_rr,x_cc,radius), (y_rr,x_cc,radius)]}
    # 2 blobs_in_PVNcells_yx_rrcc_final_dict_Ch3; {"c1":[(y_rr,x_cc,radius),(y_rr,x_cc,radius)], "c2":[(y_rr,x_cc,radius), (y_rr,x_cc,radius)]}
    # 3 blobs_mean_radius_stddev_in_PVNcells_final_dict_Ch1; {"c1":(mean_radius,std_dev), "c2":(mean_radius,std_dev)}
    # 4 blobs_mean_radius_stddev_in_PVNcells_final_dict_Ch2; {"c1":(mean_radius,std_dev), "c2":(mean_radius,std_dev)}
    # 5 blobs_mean_radius_stddev_in_PVNcells_final_dict_Ch3; {"c1":(mean_radius,std_dev), "c2":(mean_radius,std_dev)}
    # 6 maxima_in_PVNcells_yx_rrcc_final_dict_Ch1; {"c1":[(y_rr,x_cc),(y_rr,x_cc)], "c2":[(y_rr,x_cc), (y_rr,x_cc)]}
    # 7 maxima_in_PVNcells_yx_rrcc_final_dict_Ch2; {"c1":[(y_rr,x_cc),(y_rr,x_cc)], "c2":[(y_rr,x_cc), (y_rr,x_cc)]}
    # 8 maxima_in_PVNcells_yx_rrcc_final_dict_Ch3; {"c1":[(y_rr,x_cc),(y_rr,x_cc)], "c2":[(y_rr,x_cc), (y_rr,x_cc)]}
    # 9 meanintensity_PVNcells_final_dict_Ch1; {"c1":[mean], "c2":[mean]}
    # 10 meanintensity_PVNcells_final_dict_Ch2; {"c1":[mean], "c2":[mean]}
    # 11 meanintensity_PVNcells_final_dict_Ch3; {"c1":[mean], "c2":[mean]}
    # 12 area_PVNcells_yx_final_dict; {"c1":[area], "c2":[area]}
    # 13 nuclei_centroids_yx_rrcc_dict; {"c1":(y_rr,x_cc), "c2":(y_rr,x_cc)}
    # 14 area_nucleus_PVNcells_final_dict; {"c1":[area], "c2":[area]}
    """
    #get PVN masks
    if pvn_zip_files:
        l_PVN_mask, l_PVN_rr, l_PVN_cc = form_polygon(rescaled_Ch1_Ch2_Ch3[0], pvn__rois[0].coordinates())
        r_PVN_mask, r_PVN_rr, r_PVN_cc = form_polygon(rescaled_Ch1_Ch2_Ch3[0], pvn__rois[1].coordinates())
    else:
        l_PVN_mask, l_PVN_rr, l_PVN_cc = form_polygon(rescaled_Ch1_Ch2_Ch3[0], pvn__rois.coordinates())
        r_PVN_mask, r_PVN_rr, r_PVN_cc = form_polygon(rescaled_Ch1_Ch2_Ch3[0], pvn__rois.coordinates())
    
    #test functining of PVN roi opening - uncomment the cells below and provide 2 axes to show the left and right pvn mask
    # print("LEFT PVN")
    # ax2.imshow(l_PVN_mask)
    # print("RIGHT PVN")
    # ax3.imshow(r_PVN_mask)
    
    #get blobs coordinates as tuples and collect them in a list - map blobs' coordinates to their radii
    def get_blobs_coord_list_and_dict(detected_blobs):
        """
        inputs: list of blobs coordinates and radii, as dectected from function skimage.feature.blob_log
        outputs: 1) list of tuples, each tuple contains the y,x (row,column) position of each of the blob; 2) dictionary mapping each y,x coordinate of an individual blob, to its radius
        """
        blobs_coordinates_list = []
        blobs_coordinates_dict = {}
        for b in detected_blobs:
            tuple_rrcc_yx = (b[0],b[1])
            blobs_coordinates_list.append(tuple_rrcc_yx)
            if tuple_rrcc_yx not in blobs_coordinates_dict:
                blobs_coordinates_dict[tuple_rrcc_yx] = b[2]
            else:
                print("somehow a blob was present multiple times")
        return blobs_coordinates_list, blobs_coordinates_dict
    
    blobs_coordinates_list_Ch1 , blobs_coordinates_dict_Ch1 = get_blobs_coord_list_and_dict(blobs_results[0])
    blobs_coordinates_list_Ch2 , blobs_coordinates_dict_Ch2 = get_blobs_coord_list_and_dict(blobs_results[1])
    blobs_coordinates_list_Ch3 , blobs_coordinates_dict_Ch3 = get_blobs_coord_list_and_dict(blobs_results[2])
    
    
    #map cells roi names to their roi files; use nuclei' centroids to define rois' in PVN; backcalculate cells-in-PVN names from the nuclei-in-PVN names and save them (the cells-in-PVN names) in a list 
    cells_dict = {} #dictionary mapping each cell name to its roi file
    names_of_cells_in_pvn = [] #list of the names of the cells which are in the PVN
    nuclei_centroids_yx_rrcc_dict = {} #OUTPUT DICTIONARY mapping PVN-cell names to the coordinates (tuple: y,x or row,column) of their nuclei centroids
    area_nucleus_PVNcells_final_dict = {} #OUTPUT DICTIONARY mapping PVN-cell names to a list containing a single value: the area of their nuclei centroids
    
    for c in rois_file: #Iterate through cells' rois
        #identify roi type based on its name (names are "Cell #---" or "Nucleus #---" or "Cytoplasm #---" where "---" is a 3 digit number)
        c_name = list(c.name)
        hashtag_position = c_name.index("#")
        if "#" not in c_name:
            print("roi with no hashtag")
        type_of_roi = c.name[:hashtag_position-1]
        
        #if roi is a cell -> collect it in a dictionary mapping the cell name to the roi file
        if type_of_roi=="Cell":
            if c.name not in cells_dict:
                cells_dict[c.name]=c
            else:
                print("this cell was already in the cell dictionary, is it present twice?", c.name)
        
        
        #if roi is a nucleus -> find the centroid; if the centroid is in the PVN -> backcalculate corresponding cell name and save it the "names_of_cells_in_pvn" list
        elif type_of_roi=="Nucleus":
            c_mask, c_rr, c_cc = form_polygon(rescaled_Ch1_Ch2_Ch3[0], c.coordinates()) #Gets cell mask
            label_c_mask = label(c_mask) #transform cell mask in a label image
            props_c = regionprops(label_c_mask) #measure properties of the label image
            rr_y_centroid_c, cc_x_centroid_c = props_c[0]['centroid']  #Get centroid coordinates
            area_nulceus = props_c[0]['area'] #Get cell area
            value_l = l_PVN_mask[int(rr_y_centroid_c)-1, int(cc_x_centroid_c)-1] #Check if the centroid is in the left PVN
            value_r = r_PVN_mask[int(rr_y_centroid_c)-1, int(cc_x_centroid_c)-1] #Check if the centroid is in the right PVN
            if (value_l in [1,1.0]) or (value_r in [1,1.0]): #If nucleus centroid is in PVN
                #test functining of PVN intersection - uncomment the cells below and provide an ax to plot the position of centroids in pvn
                # circ = plt.Circle((int(cc_x_centroid_c)-1, int(rr_y_centroid_c)-1), 4, color='blue', linewidth=3, fill=False) #test functining of PVN intersection
                # ax3.add_patch(circ) #test functining of PVN intersection
                cell_roi_name = "Cell "+c.name[hashtag_position:] #Back-calculate cell's name
                names_of_cells_in_pvn.append(cell_roi_name) #Collect cell name in 'names_of_cells_in_pvn' list
                nuclei_centroids_yx_rrcc_dict[cell_roi_name]=(rr_y_centroid_c, cc_x_centroid_c) #Map cell name to centroid coordinates in 'nuclei_centroids_yx_rrcc_dict' dictionary
                area_nucleus_PVNcells_final_dict[cell_roi_name]=[area_nulceus] #Map cell name to nucleus area in 'area_nucleus_PVNcells_final_dict' dictionary
            #test functining of PVN intersection - uncomment the cells below and provide an ax to plot the position of centroids outside pvn
            # else: #test functining of PVN intersection
            #     circ = plt.Circle((int(cc_x_centroid_c)-1, int(rr_y_centroid_c)-1), 4, color='red', linewidth=3, fill=False) #test functining of PVN intersection
            #     ax3.add_patch(circ) #test functining of PVN intersection
    
    
    blobs_in_PVNcells_yx_rrcc_final_dict_Ch1 = {} #OUTPUT DICTIONARY mapping PVN-cell names to a list of tuples, each tuple containg in order the y_coordinate (row position), x_coordinate (column position) and radius of each blob in the cell, detected in Channel 1
    blobs_in_PVNcells_yx_rrcc_final_dict_Ch2 = {} #OUTPUT DICTIONARY mapping PVN-cell names to a list of tuples, each tuple containg in order the y_coordinate (row position), x_coordinate (column position) and radius of each blob in the cell, detected in Channel 2
    blobs_in_PVNcells_yx_rrcc_final_dict_Ch3 = {} #OUTPUT DICTIONARY mapping PVN-cell names to a list of tuples, each tuple containg in order the y_coordinate (row position), x_coordinate (column position) and radius of each blob in the cell, detected in Channel 3
    
    blobs_mean_radius_stddev_in_PVNcells_final_dict_Ch1 = {} #OUTPUT DICTIONARY mapping PVN-cell names to a tuple containg in order the mean and standard deviation of the radii of the blobs in the cell, detected in Channel 1
    blobs_mean_radius_stddev_in_PVNcells_final_dict_Ch2 = {} #OUTPUT DICTIONARY mapping PVN-cell names to a tuple containg in order the mean and standard deviation of the radii of the blobs in the cell, detected in Channel 2
    blobs_mean_radius_stddev_in_PVNcells_final_dict_Ch3 = {} #OUTPUT DICTIONARY mapping PVN-cell names to a tuple containg in order the mean and standard deviation of the radii of the blobs in the cell, detected in Channel 3
    
    maxima_in_PVNcells_yx_rrcc_final_dict_Ch1 = {} #OUTPUT DICTIONARY mapping PVN-cell names to a list of tuples, each tuple containg in order the y_coordinate (row position), x_coordinate (column position) of the maxima in the cell, detected in Channel 1
    maxima_in_PVNcells_yx_rrcc_final_dict_Ch2 = {} #OUTPUT DICTIONARY mapping PVN-cell names to a list of tuples, each tuple containg in order the y_coordinate (row position), x_coordinate (column position) of the maxima in the cell, detected in Channel 2
    maxima_in_PVNcells_yx_rrcc_final_dict_Ch3 = {} #OUTPUT DICTIONARY mapping PVN-cell names to a list of tuples, each tuple containg in order the y_coordinate (row position), x_coordinate (column position) of the maxima in the cell, detected in Channel 3
    
    meanintensity_PVNcells_final_dict_Ch1 = {} #OUTPUT DICTIONARY mapping PVN-cell names to a list containing a single value: the mean intensity of Channel 1 for that cell
    meanintensity_PVNcells_final_dict_Ch2 = {} #OUTPUT DICTIONARY mapping PVN-cell names to a list containing a single value: the mean intensity of Channel 2 for that cell
    meanintensity_PVNcells_final_dict_Ch3 = {} #OUTPUT DICTIONARY mapping PVN-cell names to a list containing a single value: the mean intensity of Channel 3 for that cell
    
    area_PVNcells_yx_final_dict = {} #OUTPUT DICTIONARY mapping PVN-cell names to a list containing a single value: the area of the cell calculated as number of pixels (see skimage.measure.regionprops)
        
    #loop through PVN cells to get their properties (number of maxima, number of blobs, area ecc...)
    for cell_n in names_of_cells_in_pvn:
        
        #get the roi file of cell in PVN - extract coordinates and cell mask - organize coordinates as a list of tuple
        cell_roi = cells_dict[cell_n]
        if cell_n != cell_roi.name:
            print("the cell name has not been associated with the correct roi")
        cell_mask, cell_rr, cell_cc = form_polygon(rescaled_Ch1_Ch2_Ch3[0], cell_roi.coordinates())
        zipped_cell_coordinates = list(zip(list(cell_rr),list(cell_cc)))
        
        #define a function to: 1) intersect cells-in-PVN coordinates with blobs coordinates (in order to get blobs which are in the cell) - 2) add the radius value to each intersected coordinate -
        # 3) modify the blobs_in_PVNcells_yx_rrcc_final_dict_Ch output dictionary (see above) by adding cell names mapped to a list of tuples, each containing in order the y_coordinate (row position), x_coordinate (column position) and radius of one of the intersected blobs -
        #4) modify the blobs_mean_radius_stddev_in_PVNcells_final_dict_Ch output dictionary (see above) by adding cell names mapped to a tuple containg in order the mean and standard deviation of the radii of the intesected blobs
        def intersect_blobs_into_cell(blobs_coordinates_list_Channel,blobs_coordinates_dict_Channel, blobs_in_PVNcells_yx_rrcc_final_dict_Channel, blobs_mean_radius_stddev_in_PVNcells_final_dict_Channel):
            intersection_blob_mask_coordinates = list(set(blobs_coordinates_list_Channel).intersection(set(zipped_cell_coordinates)))
            blobs_plus_radius_list = []
            cell_radii_list = []
            for interestected_blob in intersection_blob_mask_coordinates:
                rr_y_i_blob, cc_x_i_blob = interestected_blob[0], interestected_blob[1]
                interestected_blob_radius = blobs_coordinates_dict_Channel[(rr_y_i_blob, cc_x_i_blob)]
                yxr = (interestected_blob[0], interestected_blob[1], interestected_blob_radius) #coord_y_row, coord_x_column, radius
                blobs_plus_radius_list.append(yxr)
                cell_radii_list.append(interestected_blob_radius)
            if len(cell_radii_list)>0:
                cell_blobs_mean_radius_stddev = np.mean(cell_radii_list), np.std(cell_radii_list)
            else:
                cell_blobs_mean_radius_stddev=0.0,0.0
            blobs_in_PVNcells_yx_rrcc_final_dict_Channel[cell_n]= blobs_plus_radius_list
            blobs_mean_radius_stddev_in_PVNcells_final_dict_Channel[cell_n]= cell_blobs_mean_radius_stddev
        
        #call the function to intersect blobs of different channels with the cell in the for-loop
        intersect_blobs_into_cell(blobs_coordinates_list_Ch1,blobs_coordinates_dict_Ch1, blobs_in_PVNcells_yx_rrcc_final_dict_Ch1, blobs_mean_radius_stddev_in_PVNcells_final_dict_Ch1)
        intersect_blobs_into_cell(blobs_coordinates_list_Ch2,blobs_coordinates_dict_Ch2, blobs_in_PVNcells_yx_rrcc_final_dict_Ch2, blobs_mean_radius_stddev_in_PVNcells_final_dict_Ch2)
        intersect_blobs_into_cell(blobs_coordinates_list_Ch3,blobs_coordinates_dict_Ch3, blobs_in_PVNcells_yx_rrcc_final_dict_Ch3, blobs_mean_radius_stddev_in_PVNcells_final_dict_Ch3)
        
        #define a function to: 1) intersect cells-in-PVN coordinates with maxima coordinates (in order to get maxima which are in the cell) - 
        #2) modify the maxima_in_PVNcells_yx_rrcc_final_dict_Ch output dictionary (see above) by adding cell names mapped to a list of tuples, each containing in order the y_coordinate (row position), x_coordinate (column position) of one of the intesected maxima
        def intersect_maxima_into_cell(maxima_results_Channel, maxima_in_PVNcells_yx_rrcc_final_dict_Channel):
            yx_maxima_coordinates_list = list(zip(list(maxima_results_Channel[0]),list(maxima_results_Channel[1])))
            yx_intersection_maxima_mask_coordinates = list(set(yx_maxima_coordinates_list).intersection(set(zipped_cell_coordinates)))
            maxima_in_PVNcells_yx_rrcc_final_dict_Channel[cell_n] = yx_intersection_maxima_mask_coordinates #coord_y_row, coord_x_column
        
        #call the function to intersect maxima of different channels with the cell in the for-loop
        intersect_maxima_into_cell(maxima_results[0], maxima_in_PVNcells_yx_rrcc_final_dict_Ch1)
        intersect_maxima_into_cell(maxima_results[1], maxima_in_PVNcells_yx_rrcc_final_dict_Ch2)
        intersect_maxima_into_cell(maxima_results[2], maxima_in_PVNcells_yx_rrcc_final_dict_Ch3)
        
        
        #define a function to: 1) get intensity and area measurements of PVN-cell -
        #2) modify the meanintensity_PVNcells_final_dict_Ch output dictionary (see above) by adding cell names mapped to a list containing a single value: the mean intensity signal for the cell in the img_2_quantify input of the function -
        #3) modify the area_PVNcells_yx_final_dict output dictionary (see above) by adding cell names mapped to a list containing a single value: the area of the cell
        def get_cell_properties(img_2_quantify, meanintensity_PVNcells_final_dict_Channel, area_PVNcells_yx_final_dict_Channel):
            labeled_img = label(cell_mask)
            prop_cell = regionprops(labeled_img, intensity_image=img_2_quantify)
            mean_intensity_cell = prop_cell[0]['mean_intensity']
            area_cell = prop_cell[0]['area']
            meanintensity_PVNcells_final_dict_Channel[cell_n] = [mean_intensity_cell]
            area_PVNcells_yx_final_dict_Channel[cell_n] = [area_cell]
        
        #call the function to get area and mean intensity, for different channels, of the cell in the for-loop
        get_cell_properties(rescaled_Ch1_Ch2_Ch3[0], meanintensity_PVNcells_final_dict_Ch1, area_PVNcells_yx_final_dict)
        get_cell_properties(rescaled_Ch1_Ch2_Ch3[1], meanintensity_PVNcells_final_dict_Ch2, area_PVNcells_yx_final_dict)
        get_cell_properties(rescaled_Ch1_Ch2_Ch3[2], meanintensity_PVNcells_final_dict_Ch3, area_PVNcells_yx_final_dict)
    
    
    #test functioning of blobs and maxima intersection - NOTE: THIS FUNCTION CONTAINS A WHILE LOOP THAT CAN GO ON FORWEVER IF THERE ARE NO BLOBS OR MAXIMA DETECTED IN ANY CELL OF THE IMAGE
    def pick_a_lucky_cell(cell_name_list, blobs_maxima_in_PVNcells_yx_rrcc_final_dict_Channel):
        """
        inputs: list of cell names; dictionary associating each cell name to a list of tuples, each tuple containg in position 0 the y_coordinate (row position) and in position 1 the x_coordinate (column position) of an element (blob, maxima)
        NOTE: the function doesn't have output, it generates a picture showing a cell and as many circles as the elements (blobs, maxima) associated to the cell, the circles are centered on the coordinates of the elements
        NOTE: even though it is not required in the input, the function requires an axis to plot the cell and the elements, this is done by modifyin the "ax3" with the name of the axis to use
        IMPORTANT NOTE: the cell is picked randomly among those which have at least 1 element (blob or maxima) associatated, this means that if there is not even 1 cell which has at least 1 element, the function will loop forever
        NOTE: the function works for blobs and maxima, in principle it can be easily adapted to also plot centroids
        """
        lucky_cell = random.sample(cell_name_list, k = 1)[0]
        n_cell_blobs_maxima = len(blobs_maxima_in_PVNcells_yx_rrcc_final_dict_Channel[lucky_cell])
        while n_cell_blobs_maxima<1: #NOTE: THIS WHILE LOOP CAN GO ON FORWEVER IF THERE ARE NO BLOBS OR MAXIMA DETECTED IN ANY CELL OF THE IMAGE
            lucky_cell = random.sample(cell_name_list, k = 1)[0]
            n_cell_blobs_maxima = len(blobs_maxima_in_PVNcells_yx_rrcc_final_dict_Channel[lucky_cell])
        
        lucky_cell_roi = cells_dict[lucky_cell]
        lucky_cell_mask, lucky_cell_rr, lucky_cell_cc = form_polygon(rescaled_Ch1_Ch2_Ch3[0], lucky_cell_roi.coordinates())
        lucky_cell_blobs_maxima = blobs_maxima_in_PVNcells_yx_rrcc_final_dict_Channel[lucky_cell]
        # print(lucky_cell_blobs_maxima)
        # ax3.imshow(lucky_cell_mask) #test functining of blobs intersection, uncomment the cell and provide an axis to check blobs and maxima intersection functioning
        for lbm in lucky_cell_blobs_maxima:
            lbm_rr_y, lbm_cc_x = lbm[0], lbm[1]
            lbm_circ = plt.Circle((lbm_cc_x, lbm_rr_y), 4, color='red', linewidth=3, fill=False) #test functining of blobs intersection
            # ax3.add_patch(lbm_circ) #test functining of blobs intersection, uncomment the cell and provide an axis to check blobs and maxima intersection functioning
    
    #test functining of blobs intersection - uncomment the cell below to test blob intersection - NOTE: AN AXIS MUST BE PROVIDED BY MODIFYING THE pick_a_lucky_cell FUNCTION ABOVE; THIS FUNCTION MIGHT LOOP FOREVER (SEE DESCRIPTION ABOVE)
    # pick_a_lucky_cell(names_of_cells_in_pvn, blobs_in_PVNcells_yx_rrcc_final_dict_Ch1)
    
    #test functining of maxima intersection - uncomment the cell below to test maxima intersection - NOTE: AN AXIS MUST BE PROVIDED BY MODIFYING THE pick_a_lucky_cell FUNCTION ABOVE; THIS FUNCTION MIGHT LOOP FOREVER (SEE DESCRIPTION ABOVE)
    # pick_a_lucky_cell(names_of_cells_in_pvn, maxima_in_PVNcells_yx_rrcc_final_dict_Ch1)
    
    tuple_output = (blobs_in_PVNcells_yx_rrcc_final_dict_Ch1, blobs_in_PVNcells_yx_rrcc_final_dict_Ch2, blobs_in_PVNcells_yx_rrcc_final_dict_Ch3, blobs_mean_radius_stddev_in_PVNcells_final_dict_Ch1, blobs_mean_radius_stddev_in_PVNcells_final_dict_Ch2, blobs_mean_radius_stddev_in_PVNcells_final_dict_Ch3, maxima_in_PVNcells_yx_rrcc_final_dict_Ch1, maxima_in_PVNcells_yx_rrcc_final_dict_Ch2, maxima_in_PVNcells_yx_rrcc_final_dict_Ch3, meanintensity_PVNcells_final_dict_Ch1, meanintensity_PVNcells_final_dict_Ch2, meanintensity_PVNcells_final_dict_Ch3, area_PVNcells_yx_final_dict, nuclei_centroids_yx_rrcc_dict, area_nucleus_PVNcells_final_dict)
    
    return tuple_output



def form_principal_dataframe(experiment_n, sex_n, mouse_n, slice_pos_n, results_tuple):
    """
    spreads cell measurements from GetCiPVNwIntMaxBlobs function on a dictionary which can be used to generate a dataframe where rows are individual cells and columns are cells following properties:
    experiment, sex, mouse, slice_position, cell name, number of blobs in Ch1, number of blobs in Ch2, number of blobs in Ch3, mean radius of blobs in Ch1, standard deviation of blobs in Ch1, mean radius of blobs in Ch2, standard deviation of blobs in Ch2,
    mean radius of blobs in Ch3, standard deviation of blobs in Ch3, number of maxima in Ch1, number of maxima in Ch2, number of maxima in Ch3, mean signal intensity in Ch1, mean signal intensity in Ch2, mean signal intensity in Ch3,
    area of cell (pixel number), y coordinate of cell nucleus centroid (used to assign the cell to PVN), x coordinate of cell nucleus centroid (used to assign the cell to PVN), area of cell nucleus
    
    inputs: name of the experiment (string), sex (string), mouse name (string), slice position (string), tuple of results from GetCiPVNwIntMaxBlobs
    results of GetCiPVNwIntMaxBlobs function are the following, I also indicate, per each measurement how it is organized
    # 0 blobs_in_PVNcells_yx_rrcc_final_dict_Ch1; {"c1":[(y_rr,x_cc,radius),(y_rr,x_cc,radius)], "c2":[(y_rr,x_cc,radius), (y_rr,x_cc,radius)]}
    # 1 blobs_in_PVNcells_yx_rrcc_final_dict_Ch2; {"c1":[(y_rr,x_cc,radius),(y_rr,x_cc,radius)], "c2":[(y_rr,x_cc,radius), (y_rr,x_cc,radius)]}
    # 2 blobs_in_PVNcells_yx_rrcc_final_dict_Ch3; {"c1":[(y_rr,x_cc,radius),(y_rr,x_cc,radius)], "c2":[(y_rr,x_cc,radius), (y_rr,x_cc,radius)]}
    # 3 blobs_mean_radius_stddev_in_PVNcells_final_dict_Ch1; {"c1":(mean_radius,std_dev), "c2":(mean_radius,std_dev)}
    # 4 blobs_mean_radius_stddev_in_PVNcells_final_dict_Ch2; {"c1":(mean_radius,std_dev), "c2":(mean_radius,std_dev)}
    # 5 blobs_mean_radius_stddev_in_PVNcells_final_dict_Ch3; {"c1":(mean_radius,std_dev), "c2":(mean_radius,std_dev)}
    # 6 maxima_in_PVNcells_yx_rrcc_final_dict_Ch1; {"c1":[(y_rr,x_cc),(y_rr,x_cc)], "c2":[(y_rr,x_cc), (y_rr,x_cc)]}
    # 7 maxima_in_PVNcells_yx_rrcc_final_dict_Ch2; {"c1":[(y_rr,x_cc),(y_rr,x_cc)], "c2":[(y_rr,x_cc), (y_rr,x_cc)]}
    # 8 maxima_in_PVNcells_yx_rrcc_final_dict_Ch3; {"c1":[(y_rr,x_cc),(y_rr,x_cc)], "c2":[(y_rr,x_cc), (y_rr,x_cc)]}
    # 9 meanintensity_PVNcells_final_dict_Ch1; {"c1":[mean], "c2":[mean]}
    # 10 meanintensity_PVNcells_final_dict_Ch2; {"c1":[mean], "c2":[mean]}
    # 11 meanintensity_PVNcells_final_dict_Ch3; {"c1":[mean], "c2":[mean]}
    # 12 area_PVNcells_yx_final_dict; {"c1":[area], "c2":[area]}
    # 13 nuclei_centroids_yx_rrcc_dict; {"c1":(y_rr,x_cc), "c2":(y_rr,x_cc)}
    # 14 area_nucleus_PVNcells_final_dict; {"c1":[area], "c2":[area]}
    
    outputs: dictionary: per each feature/measured_property, the feature/measured_property, as a string, is associated to a list of length equal to number_of_analyzed_cells and values equal to the corresponding feature/measured_property. Cell identity is maintained as the position in each list.
    """
    
    final_dict = {}
    
    for c in results_tuple[0]:
        if 'experiment' not in final_dict:
            final_dict['experiment']=[experiment_n]
        else:
            final_dict['experiment'].append(experiment_n)
            
        if 'sex' not in final_dict:
            final_dict['sex']=[sex_n]
        else:
            final_dict['sex'].append(sex_n)
            
        if 'mouse' not in final_dict:
            final_dict['mouse']=[mouse_n]
        else:
            final_dict['mouse'].append(mouse_n)
            
        if 'slice_position' not in final_dict:
            final_dict['slice_position']=[slice_pos_n]
        else:
            final_dict['slice_position'].append(slice_pos_n)
            
        if 'cell' not in final_dict:
            final_dict['cell']=[c]
        else:
            final_dict['cell'].append(c)
            
        if 'blobs_number_Ch1' not in final_dict:
            final_dict['blobs_number_Ch1']=[len(results_tuple[0][c])]
        else:
            final_dict['blobs_number_Ch1'].append(len(results_tuple[0][c]))
            
        if 'blobs_number_Ch2' not in final_dict:
            final_dict['blobs_number_Ch2']=[len(results_tuple[1][c])]
        else:
            final_dict['blobs_number_Ch2'].append(len(results_tuple[1][c]))
            
        if 'blobs_number_Ch3' not in final_dict:
            final_dict['blobs_number_Ch3']=[len(results_tuple[2][c])]
        else:
            final_dict['blobs_number_Ch3'].append(len(results_tuple[2][c]))
                
        if 'mean_blob_radius_Ch1' not in final_dict:
            final_dict['mean_blob_radius_Ch1']=[results_tuple[3][c][0]]
        else:
            final_dict['mean_blob_radius_Ch1'].append(results_tuple[3][c][0])
            
        if 'std_dev_blob_radius_Ch1' not in final_dict:
            final_dict['std_dev_blob_radius_Ch1']=[results_tuple[3][c][1]]
        else:
            final_dict['std_dev_blob_radius_Ch1'].append(results_tuple[3][c][1])
            
        if 'mean_blob_radius_Ch2' not in final_dict:
            final_dict['mean_blob_radius_Ch2']=[results_tuple[4][c][0]]
        else:
            final_dict['mean_blob_radius_Ch2'].append(results_tuple[4][c][0])
            
        if 'std_dev_blob_radius_Ch2' not in final_dict:
            final_dict['std_dev_blob_radius_Ch2']=[results_tuple[4][c][1]]
        else:
            final_dict['std_dev_blob_radius_Ch2'].append(results_tuple[4][c][1])
            
        if 'mean_blob_radius_Ch3' not in final_dict:
            final_dict['mean_blob_radius_Ch3']=[results_tuple[5][c][0]]
        else:
            final_dict['mean_blob_radius_Ch3'].append(results_tuple[5][c][0])
            
        if 'std_dev_blob_radius_Ch3' not in final_dict:
            final_dict['std_dev_blob_radius_Ch3']=[results_tuple[5][c][1]]
        else:
            final_dict['std_dev_blob_radius_Ch3'].append(results_tuple[5][c][1])
            
        if 'maxima_number_Ch1' not in final_dict:
            final_dict['maxima_number_Ch1']=[len(results_tuple[6][c])]
        else:
            final_dict['maxima_number_Ch1'].append(len(results_tuple[6][c]))
            
        if 'maxima_number_Ch2' not in final_dict:
            final_dict['maxima_number_Ch2']=[len(results_tuple[7][c])]
        else:
            final_dict['maxima_number_Ch2'].append(len(results_tuple[7][c]))
            
        if 'maxima_number_Ch3' not in final_dict:
            final_dict['maxima_number_Ch3']=[len(results_tuple[8][c])]
        else:
            final_dict['maxima_number_Ch3'].append(len(results_tuple[8][c]))
            
        if 'mean_intensity_Ch1' not in final_dict:
            final_dict['mean_intensity_Ch1']=[results_tuple[9][c][0]]
        else:
            final_dict['mean_intensity_Ch1'].append(results_tuple[9][c][0])
            
        if 'mean_intensity_Ch2' not in final_dict:
            final_dict['mean_intensity_Ch2']=[results_tuple[10][c][0]]
        else:
            final_dict['mean_intensity_Ch2'].append(results_tuple[10][c][0])
            
        if 'mean_intensity_Ch3' not in final_dict:
            final_dict['mean_intensity_Ch3']=[results_tuple[11][c][0]]
        else:
            final_dict['mean_intensity_Ch3'].append(results_tuple[11][c][0])
            
        if 'area_cell' not in final_dict:
            final_dict['area_cell']=[results_tuple[12][c][0]]
        else:
            final_dict['area_cell'].append(results_tuple[12][c][0])
            
        if 'nucleus_centroid_row_y' not in final_dict:
            final_dict['nucleus_centroid_row_y']=[results_tuple[13][c][0]]
        else:
            final_dict['nucleus_centroid_row_y'].append(results_tuple[13][c][0])
            
        if 'nucleus_centroid_col_x' not in final_dict:
            final_dict['nucleus_centroid_col_x']=[results_tuple[13][c][1]]
        else:
            final_dict['nucleus_centroid_col_x'].append(results_tuple[13][c][1])
        
        if 'area_nucleus' not in final_dict:
            final_dict['area_nucleus']=[results_tuple[14][c][0]]
        else:
            final_dict['area_nucleus'].append(results_tuple[14][c][0])
    
    return final_dict



In [10]:
#iterate processing on images

def pipeline_processing(img):
    #form paths for files to be used (images and rois)
    orig_img_path = os.path.join(home_directory, os.path.join(step1_path_output, organized_dictionary[img]['original_image']))
    pvn_roi_path = os.path.join(home_directory, os.path.join(step1_path_output, organized_dictionary[img]['rois_pvn']))
    cells_roi_path = os.path.join(home_directory, os.path.join(step3_path_output, organized_dictionary[img]['cell_rois']))
    
    #open image and split channels
    orig_img = tifffile.imread(orig_img_path)
    ch1_img = orig_img[0,:,:]
    ch2_img = orig_img[1,:,:]
    ch3_img = orig_img[2,:,:]
    ch4_img = orig_img[3,:,:] #DAPI
    
    #intensity histogram rescaling
    equalized_np_ch1 = my_intensity_rescaling(ch1_img, min_percentil, max_percentil) #NOTE: the rescaling is done using the mode value as minimum, it is assumed the mode will reflect the tissue background
    equalized_np_ch2 = my_intensity_rescaling(ch2_img, min_percentil, max_percentil) #NOTE: the rescaling is done using the mode value as minimum, it is assumed the mode will reflect the tissue background
    equalized_np_ch3 = my_intensity_rescaling(ch3_img, min_percentil, max_percentil) #NOTE: the rescaling is done using the mode value as minimum, it is assumed the mode will reflect the tissue background
    
    #detect blobs in each channel
    blobs_log_Ch1 = blob_log(equalized_np_ch1, min_sigma=min_sigma_Ch1, max_sigma=max_sigma_Ch1, num_sigma=num_sigma_Ch1, threshold=threshold_Ch1, overlap=overlap_Ch1)
    blobs_log_Ch2 = blob_log(equalized_np_ch2, min_sigma=min_sigma_Ch2, max_sigma=max_sigma_Ch2, num_sigma=num_sigma_Ch2, threshold=threshold_Ch2, overlap=overlap_Ch2)
    blobs_log_Ch3 = blob_log(equalized_np_ch3, min_sigma=min_sigma_Ch3, max_sigma=max_sigma_Ch3, num_sigma=num_sigma_Ch3, threshold=threshold_Ch3, overlap=overlap_Ch3)
    
    #maxima detection
    #maxima detection - rescaling of images as int16 with range 0,255
    rescaled_Ch1 = img_as_ubyte(equalized_np_ch1)
    rescaled_Ch2 = img_as_ubyte(equalized_np_ch2)
    rescaled_Ch3 = img_as_ubyte(equalized_np_ch3)
    #detect maxima in each channel
    maxima_Ch1 = my_find_maxima(rescaled_Ch1, maxima_threshold_Ch1)
    maxima_Ch2 = my_find_maxima(rescaled_Ch2, maxima_threshold_Ch2)
    maxima_Ch3 = my_find_maxima(rescaled_Ch3, maxima_threshold_Ch3)
    
    #open PVN and Cells rois
    pvn_rois = ImagejRoi.fromfile(pvn_roi_path)
    cells_rois = ImagejRoi.fromfile(cells_roi_path)
    
    #Select/measure properties of cells in the PVNs
    if organized_dictionary[img]['rois_pvn'][-4:]=='.roi':
        measure_results = GetCiPVNwIntMaxBlobs((equalized_np_ch1, equalized_np_ch2, equalized_np_ch3), pvn_rois, cells_rois, (blobs_log_Ch1,blobs_log_Ch2,blobs_log_Ch3), (maxima_Ch1,maxima_Ch2,maxima_Ch3), False)
        print(img, "individual pvn file")
    else:
        measure_results = GetCiPVNwIntMaxBlobs((equalized_np_ch1, equalized_np_ch2, equalized_np_ch3), pvn_rois, cells_rois, (blobs_log_Ch1,blobs_log_Ch2,blobs_log_Ch3), (maxima_Ch1,maxima_Ch2,maxima_Ch3), True)
    
    
    #get experiment, sex, mouse and slice_position information
    
    #Define a function to slice strings according to indicated delimeters. A delimiter indicated as None is interpreted as no delimiter should be used. This allows to include the beginning and the end of the string in the slicing process
    #The function is used to get the experiment sex, mouse and slice_position names, according to the slice delimiters defined in the hyperparameter settings
    def get_name_of_interest(string_2_slice, slice_delimiter_1, slice_delimiter_2):
        if (slice_delimiter_1==None and slice_delimiter_2==None):
            name_of_interest = string_2_slice[:]
        elif (slice_delimiter_1==None and slice_delimiter_2!=None):
            name_of_interest = string_2_slice[:slice_delimiter_2]
        elif slice_delimiter_1!=None and slice_delimiter_2==None:
            name_of_interest = string_2_slice[slice_delimiter_1:]
        else:
            name_of_interest = string_2_slice[slice_delimiter_1:slice_delimiter_2]
        return name_of_interest
    
    #Get experiment name
    if isinstance(experiment_position, str):
        experiment = experiment_position
    else:
        experiment = get_name_of_interest(home_directory, experiment_position[0],experiment_position[1]) #Get experiment name
    
    #Get sex name
    if isinstance(sex_position, str):
        sex = sex_position
    else:
        sex = get_name_of_interest(img, sex_position[0],sex_position[1]) #Get Sex name
    
    #Get mouse name
    if isinstance(mouse_position, str):
        mouse = mouse_position
    else:
        mouse = get_name_of_interest(img, mouse_position[0],mouse_position[1]) #Get mouse name
    
    #Get slice position
    if isinstance(slice_position, str):
        slice_pos = slice_position
    else:
        slice_pos = get_name_of_interest(img, slice_position[0],slice_position[1]) #Get slice_position name
    
    #Generate final dictionary
    my_final_dict = form_principal_dataframe(experiment, sex, mouse, slice_pos, measure_results)
    
    #Create final dataframe
    experiment_results_dataframe = pd.DataFrame.from_dict(my_final_dict)
    
    return experiment_results_dataframe


#Iterate through all images and collect results in a list
results_dataframe_list = []
for img_n in organized_dictionary:
    print("-------------------------")
    print("working on: ", img_n)
    img_results_df = pipeline_processing(img_n)
    results_dataframe_list.append(img_results_df)
    print("finished: ", img_n)

#Concatenate all dataframes in a single, final file
results_dataframe = pd.concat(results_dataframe_list,axis=0)

# print(results_dataframe)

#Save the result
output_name = home_directory[-11:] + "_results.csv"
print(output_name)
results_dataframe.to_csv(os.path.join(home_directory, output_name), index=False)

print("Finished")


-------------------------
working on:  M_07_fron_MAX_cut
finished:  M_07_fron_MAX_cut
-------------------------
working on:  F_06_fron_MAX_cut
finished:  F_06_fron_MAX_cut
-------------------------
working on:  F_12_fron_MAX_cut
finished:  F_12_fron_MAX_cut
-------------------------
working on:  F_20_back_MAX_cut
finished:  F_20_back_MAX_cut
-------------------------
working on:  M_04_back_MAX_cut
finished:  M_04_back_MAX_cut
-------------------------
working on:  F_02_fron_MAX_cut
finished:  F_02_fron_MAX_cut
-------------------------
working on:  M_03_fron_MAX_cut
finished:  M_03_fron_MAX_cut
-------------------------
working on:  F_13_fron_MAX_cut
finished:  F_13_fron_MAX_cut
-------------------------
working on:  F_17_fron_MAX_cut
finished:  F_17_fron_MAX_cut
-------------------------
working on:  M_09_fron_MAX_cut
finished:  M_09_fron_MAX_cut
-------------------------
working on:  M_05_back_MAX_cut
finished:  M_05_back_MAX_cut
-------------------------
working on:  M_02_fron_MAX_c